In [ ]:
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import cv2
import re
import string
import os
from tqdm import tqdm

#Model management
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import torch
import joblib

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

#image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [ ]:
try:
    os.mkdir("crop_images")
except:
    pass

In [ ]:
def get_img(idim,floderpath):
    imtarget = floderpath + idim
    impath = imtarget + "/image.png"
    dfpath = imtarget + "/gt_text.json"
    
    im = cv2.imread(impath)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY )
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    df = pd.read_json(dfpath)
    num = df.shape[0]
    
    image_list = []
    filename_list = []
    fontFamily_list = []
    fontSize_list = []
    fontStyle_list = []
    fontVarient_list = []
    fontWeight_list = []
    h_list = []
    w_list = []
    text_list= []
    parentId_list=[]
    
    for i in range(num):
        if df["text"][i].strip("_.)(][}{\\/:") != "":
            x  = df["rect"][i]["x"]
            y  = df["rect"][i]["y"]
            width  = df["rect"][i]["width"]
            height  = df["rect"][i]["height"]
            if y > im.shape[0] or x > im.shape[1]:
                pass
            else:
                image = np.array(im[int(y)-3:int(y+height)+3,int(x)-3:int(x+width)+3])
                imagename = idim +"_" + df["parentId"][i]
                #image_list.append(image)
                cv2.imwrite(f"./crop_images/{imagename}.png",image)

                #append
                image_list.append(imagename)
                filename_list.append(f"{imagename}.png")
                fontFamily_list.append(df["style"][i]["fontFamily"])
                fontSize_list.append(df["style"][i]["fontSize"])
                fontStyle_list.append(df["style"][i]["fontStyle"])
                fontWeight_list.append(df["style"][i]["fontWeight"])
                h_list.append(height)
                w_list.append(width)
                text_list.append(df["text"][i])
                parentId_list.append(df["parentId"][i])
        

    #df["images"] = np.array(image_list)
    dfn = pd.DataFrame(image_list,columns =['imagename'])
    dfn["file"] = pd.Series(filename_list)   
    dfn["parentId"] = pd.Series(parentId_list)
    dfn["text"] = pd.Series(text_list)
    dfn["fontFamily"] = pd.Series(fontFamily_list)
    dfn["fontSize"] = pd.Series(fontSize_list)
    dfn["fontStyle"] = pd.Series(fontStyle_list)
    dfn["fontWeight"] = pd.Series(fontWeight_list)
    dfn["h"] = pd.Series(h_list)
    dfn["w"] = pd.Series(w_list)
    dfn["Id"] = idim
    return dfn

```
floderpath = "../input/super-ai-engineer-2021-font-recognition/train/set1/"
filelist = os.listdir(floderpath)
df1 = get_img(filelist[0],floderpath)
for i in tqdm(range(len(filelist))):
    idim = filelist[i]
    if idim != filelist[0]:
        df3 = get_img(idim,floderpath)
        df1 = df1.append(df3,ignore_index=True)
```

In [ ]:
floderpath = "../input/super-ai-engineer-2021-font-recognition/train/set1/"
filelist = os.listdir(floderpath)
df1 = get_img(filelist[0],floderpath)
for i in tqdm(range(len(filelist))):
    idim = filelist[i]
    if idim != filelist[0]:
        df3 = get_img(idim,floderpath)
        df1 = df1.append(df3,ignore_index=True)

In [ ]:
floderpath = "../input/super-ai-engineer-2021-font-recognition/train/set2/"
filelist = os.listdir(floderpath)
for i in tqdm(range(len(filelist))):
    idim = filelist[i]
    df3 = get_img(idim,floderpath)
    df1 = df1.append(df3,ignore_index=True)

In [ ]:
df1.to_csv("df.csv",index=False)

In [ ]:
floderpath = "./crop_images"
filelist = os.listdir(floderpath)

In [ ]:
df1.shape[0]

In [ ]:
len(filelist)